# Daily Stock Data Download & Performance Report

This notebook is designed to be run daily to:
1. Download the latest stock price data
2. Generate performance summaries
3. Save data to CSV for historical tracking
4. Display key metrics and visualizations

**Last Updated:** Run this daily to keep your data current

## Setup & Configuration

In [ ]:
# Import required libraries
from functions import *
from datetime import datetime
import os

# Create data directory if it doesn't exist
data_dir = 'stock_data'
os.makedirs(data_dir, exist_ok=True)

# Get today's date for file naming
today = datetime.now().strftime('%Y-%m-%d')
print(f"📅 Running daily download for: {today}")

## Define Your Watchlist

Edit this list to track your preferred stocks and ETFs.

In [ ]:
# Your portfolio/watchlist tickers
my_tickers = [
    'AAPL',   # Apple
    'MSFT',   # Microsoft
    'GOOGL',  # Alphabet
    'AMZN',   # Amazon
    'NVDA',   # NVIDIA
    'TSLA',   # Tesla
    'META',   # Meta
]

# Benchmark ETFs (automatically added)
benchmarks = ['SPY', 'VOO', 'QQQ']

all_tickers = my_tickers + benchmarks
print(f"📊 Tracking {len(my_tickers)} stocks and {len(benchmarks)} benchmarks")

## Download Latest Price Data

In [ ]:
# Download data for multiple timeframes
print("⬇️ Downloading data...\n")

# Get 1 day of data (latest prices)
latest_data = yf.download(all_tickers, period='5d', progress=False)

# Extract latest prices
if 'Adj Close' in latest_data:
    latest_prices = latest_data['Adj Close'].iloc[-1]
else:
    latest_prices = latest_data['Close'].iloc[-1]

# Convert to DataFrame and save
latest_df = pd.DataFrame({
    'Ticker': latest_prices.index,
    'Price': latest_prices.values,
    'Date': today
})

print("\n✅ Latest Prices:")
print(latest_df.to_string(index=False))

# Save to CSV (append mode)
csv_file = f"{data_dir}/daily_prices.csv"
if os.path.exists(csv_file):
    latest_df.to_csv(csv_file, mode='a', header=False, index=False)
    print(f"\n📝 Appended to {csv_file}")
else:
    latest_df.to_csv(csv_file, index=False)
    print(f"\n📝 Created new file: {csv_file}")

## Performance Summary - Multiple Timeframes

In [ ]:
# 1 Week Performance
print("\n" + "="*80)
print("📈 1-WEEK PERFORMANCE")
print("="*80)
summary_1w = generate_performance_summary(my_tickers, period='5d', benchmark='SPY')

In [ ]:
# 1 Month Performance
summary_1m = generate_performance_summary(my_tickers, period='1mo', benchmark='SPY')

In [ ]:
# 3 Month Performance
summary_3m = generate_performance_summary(my_tickers, period='3mo', benchmark='SPY')

In [ ]:
# Year-to-Date Performance
summary_ytd = generate_performance_summary(my_tickers, period='ytd', benchmark='SPY')

In [ ]:
# 1 Year Performance
summary_1y = generate_performance_summary(my_tickers, period='1y', benchmark='SPY')

## Save Performance Summaries

In [ ]:
# Save each timeframe summary
summaries = {
    '1week': summary_1w,
    '1month': summary_1m,
    '3month': summary_3m,
    'ytd': summary_ytd,
    '1year': summary_1y
}

for period, df in summaries.items():
    filename = f"{data_dir}/performance_{period}_{today}.csv"
    df.to_csv(filename)
    print(f"💾 Saved {filename}")

## Normalized Price Chart - YTD

In [ ]:
# Plot normalized performance
normalized = download_and_plot_stock_data(my_tickers, period='ytd')

## Top Performers & Laggards

In [ ]:
# Identify top performers and laggards for 1Y period
print("\n" + "="*80)
print("🏆 TOP 3 PERFORMERS (1 Year)")
print("="*80)
print(summary_1y.head(3)[['Total Return (%)', 'Annualized Return (%)', 'vs Benchmark (%)']].to_string())

print("\n" + "="*80)
print("📉 BOTTOM 3 PERFORMERS (1 Year)")
print("="*80)
print(summary_1y.tail(3)[['Total Return (%)', 'Annualized Return (%)', 'vs Benchmark (%)']].to_string())

print("\n")

## Daily Summary Report

In [ ]:
# Create a comprehensive daily summary
print("\n" + "="*80)
print(f"📋 DAILY SUMMARY REPORT - {today}")
print("="*80)

# Count winners vs losers (1 day)
winners_1d = (summary_1w['Total Return (%)'] > 0).sum()
losers_1d = (summary_1w['Total Return (%)'] < 0).sum()

print(f"\n📊 Market Breadth (5 days):")
print(f"   Winners: {winners_1d} | Losers: {losers_1d}")

# Best and worst performers
best = summary_1w['Total Return (%)'].idxmax()
worst = summary_1w['Total Return (%)'].idxmin()

print(f"\n🌟 Best Performer: {best} ({summary_1w.loc[best, 'Total Return (%)']}%)")
print(f"⚠️  Worst Performer: {worst} ({summary_1w.loc[worst, 'Total Return (%)']}%)")

# Benchmark comparison
avg_return = summary_1w['Total Return (%)'].mean()
spy_return = summary_1w.loc['SPY', 'Total Return (%)'] if 'SPY' in summary_1w.index else 0

print(f"\n📈 Average Return (5d): {avg_return:.2f}%")
print(f"📊 SPY Return (5d): {spy_return:.2f}%")
print(f"\n{'='*80}\n")

## Next Steps

1. Review the performance summaries above
2. Check the saved CSV files in the `stock_data/` directory
3. Run this notebook again tomorrow to track daily changes
4. Modify the `my_tickers` list as needed

**Tip:** Set up a daily reminder to run this notebook for consistent tracking!